# Sign language classification on sign-language-mnist data

In [6]:
## Import Libraries
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import csv
import numpy as np

In [7]:
## Define Parameters
EPOCHS = 15
NUM_ROWS = 28
NUM_COLS = 28
ROW_LEN = NUM_ROWS * NUM_COLS + 1

In [10]:
## Function to load csv data
def load_csv_data(filename):
    data = []
    labels = []
    with open(filename) as csvFile:
        csvReader = csv.reader(csvFile)
        first_line = True
        for row in csvReader:
            if first_line:
                # print("Ignoring first line")
                first_line = False
            else:
                labels.append(row[0])
                data_row = row[1:ROW_LEN]
                data_row = np.array_split(data_row, NUM_ROWS)
                data.append(data_row)
        data = np.array(data).astype('float')
        labels = np.array(labels).astype('float')
    return data, labels


In [11]:
## Load Data
data_dir = "D://Datasets//sign_lang_mnist//"
train_file = data_dir + "sign_mnist_train.csv"
test_file = data_dir + "sign_mnist_test.csv"

train_data, train_labels = load_csv_data(train_file)
test_data, test_labels = load_csv_data(test_file)

print(train_data.shape)
print(test_data.shape)
print(train_labels.shape)
print(test_labels.shape)

(27455, 28, 28)
(7172, 28, 28)
(27455,)
(7172,)


In [19]:
## Expand data dimensions to support with Keras
train_data = np.expand_dims(train_data, axis=3)
test_data = np.expand_dims(test_data, axis=3)

In [20]:
## Define ImageDataGenerator
train_gen = ImageDataGenerator(rescale=1/255.,
                               rotation_range=40,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               zoom_range=0.2,
                               shear_range=0.2,
                               horizontal_flip=True)
val_gen = ImageDataGenerator(rescale=1/255.)

In [21]:
## Define Model
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(256, (3,3), activation='relu', input_shape=(28,28,1)),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dense(26, activation='softmax')])

In [22]:
## Compile Model
model.compile(optimizer=RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['acc'])

In [23]:
## Define Callback
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs={}):
        if (logs.get('acc') > 0.99):
            print ("\nAccuracy reached 99%. Stopping operations.")
            self.model.stop_training = True
callback = myCallback()

In [ ]:
## Train Model
history = model.fit_generator(train_gen.flow(train_data, train_labels, batch_size=32),
                              epochs=EPOCHS,
                              validation_data=val_gen.flow(test_data, test_labels, batch_size=32),
                              callbacks=[callback],
                              verbose=1)

Instructions for updating:
Use tf.cast instead.


In [ ]:
## Plot Loss and Accuracy
train_acc = history.history['acc']
val_acc = history.history['val_acc']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(train_acc))

plt.plot(epochs, train_acc, 'r', label='train_acc')
plt.plot(epochs, val_acc, 'b', label='val_acc')
plt.title('Train and Val accuracies')
plt.xlabel('Epochs')
plt.ylabel('Accuracies')
plt.legend
plt.figure

plt.plot(epochs, train_loss, 'r', label='train_loss')
plt.plot(epochs, val_loss, 'b', label='val_loss')
plt.title('Train and Val losses')
plt.xlabel('Epochs')
plt.ylabel('Accuracies')
plt.legend

plt.show()

In [ ]:
## Evaluate Model
model.evaluate(test_data, test_labels)